## Import and dataserver setup

In [ ]:
## Import of required modules

import time
import zhinst.ziPython as zi
import zhinst.utils
import numpy as np
from scipy.linalg import expm
import json
import random
import matplotlib.pyplot as plt

In [ ]:
## Define the parameters of your instrument and dataserver

# dataserver IP address - may be localhost or any IP running the LabOne dataserver
# dataserver_host = 'localhost'
dataserver_host = 'your_dataserver_IP_here'

# HDAWG device name
# dev_hd = 'devYYYY'
dev_hd = 'your_device_ID_here'

In [ ]:
# connect to ZI dataserver 
daq = zi.ziDAQServer(dataserver_host, 8004, 6)

# check for required LabOne version
version = daq.getInt(f'/zi/about/revision')
if version < 200702701:
    raise Exception(f"This script requires LabOne version 20.07.2701 or higher.")

## Helper class for controlling the HDAWG

In [ ]:
class HDAWG_Core():
    
    def __init__(self, daq, device, awg_index=0, connect_type = '1gbe', iq_modulation=True, osc_control=1, wait=0.005):
        """Configure the device. Select mode of 2 channels in each group
        
        Parameters
        ----------
        daq : ziDAQServer 
            The DAQ connection
        device : str
            The serial of the HDAWG
        awg_index: int
            The index of the AWG core
        """
        self.daq = daq
        self.device = device
        self.awg_index = awg_index
        
        # waiting interval for polling of internal states
        self.wait = wait

        self.daq.connectDevice(device, connect_type)

        # Configure 4x2 mode
        self.daq.setString(f'/{self.device}/system/awg/channelgrouping', 'groups_of_2')
    
        #IQ modulation
        self.iq_modulation = iq_modulation
        
        # set oscillator control setting
        self.setOscControl(osc_control)
        
        # Setup AWG module
        self.awg_module = daq.awgModule()
        self.awg_module.set('device', device)
        self.awg_module.set('index', awg_index)
        
        # Execute commands
        self.awg_module.execute()
        

    def config(self, program, ct=None, waves=None):
        """Configure the HDAWG with a seqC program and optional a command table and waveform table
        
        Parameters
        ----------
        program: str
            The seqc program
        ct: dict
            The Command Table, as list
        waves: list
            List of the waveforms
        """
        ## Configure AWG
        # Stop AWG
        self.daq.setInt(f'/{self.device}/awgs/{self.awg_index}/enable', 0)
        # Send sequence
        self.compile_seqc(program)

        # Run AWG program only once
        self.daq.setInt(f'/{self.device}/awgs/{self.awg_index}/single', 1)

        # Enable channel outputs
        self.daq.setInt(f'/{self.device}/sigouts/{self.awg_index*2}/on', 1)
        self.daq.setInt(f'/{self.device}/sigouts/{self.awg_index*2+1}/on', 1)

        # upload AWG waveform data
        if waves is not None:
            for i, wave in enumerate(waves):
                wave_raw = zhinst.utils.convert_awg_waveform(wave[0],wave[1])
                self.daq.setVector(f'/{self.device}/awgs/{self.awg_index}/waveform/waves/{i}', wave_raw)

        # upload the command table
        if ct is not None:
            self.load_ct(ct)
            
        
    def setUserReg(self, register, value):
        """Set the value of a single UserRegister
        
        Parameters
        ----------
        register: int
            index of the user register to be set
        value: int
            value of the user register to be set
        """
        self.daq.setInt(f'/{self.device:s}/awgs/{self.awg_index}/userregs/{register:d}', value)
        

    def setUserRegisters(self, registers):
        """Set the value of multiple UserRegister
        
        Parameters
        ----------
        registers: list
            list of indices and values for the user registers to be read back
        """
        sets = [(f'/{self.device:s}/awgs/{self.awg_index}/userregs/{reg[0]:d}', reg[1]) for reg in registers]
        self.daq.set(sets)
        

    def getUserReg(self, register):
        """Read back the value of a UserRegister
        
        Parameters
        ----------
        register: int
            the index of the user register to be read back
        """
        self.daq.getInt(f'/{self.device:s}/awgs/{self.awg_index}/userregs/{register:d}')
        

    def setHold(self, hold=1):
        """enable / disable last sample hold
        
        Parameters
        ----------
        hold: int
            enable or disable last sample hold
        """
        self.daq.setInt(f'/{self.device:s}/awgs/{self.awg_index}/outputs/0/hold', hold)
        self.daq.setInt(f'/{self.device:s}/awgs/{self.awg_index}/outputs/1/hold', hold)
        

    def setOscControl(self, enable):
        """enable / disable control of digital oscillator frequencies in sequencer program
        
        Parameters
        ----------
        enable: int
            enable or disable software control
        """
        self.daq.setInt(f'/{self.device:s}/system/awg/oscillatorcontrol', enable)
        
    
    def run(self, wait=False):
        """run the loaded seqC program
        
        Parameters
        ----------
        wait: bool
            wait for AWG to return ready state
        """
        enable = f'/{self.device:s}/awgs/{self.awg_index}/enable'
        self.daq.syncSetInt(enable, 1)
        if wait:
            while(self.daq.getInt(enable) == 1):
                time.sleep(self.wait)
                
                
    def stop(self):
        """stop the AWG core
        """
        enable = f'/{self.device:s}/awgs/{self.awg_index}/enable'
        self.daq.setInt(enable, 0)
        

    def compile_seqc(self, program):
        """Compile and send a sequencer program to the device
        
        Parameters
        ----------
        program: str
            The seqc program
        """
        # Compile program
        self.awg_module.set('compiler/sourcestring', program)
        while self.awg_module.getInt('compiler/status') == -1:
            time.sleep(self.wait)
        if self.awg_module.getInt('compiler/status') != 0:
            raise Exception("Failed to compile program.")
        
        # Upload program
        while (self.awg_module.getDouble('progress') < 1.0) and (self.awg_module.getInt('elf/status') != 1):
            time.sleep(self.wait)
        if self.awg_module.getInt('elf/status') == 1:
            raise Exception("Failed to upload program.")
            
            
    def load_waves(self, waves):
        """uploads a set of waveforms to the waveform memory
        
        Parameters
        ----------
        waves: list
            list of two-channel waveforms, given as [[I], [Q]]
        """
        #load AWG waveforms
        for i, wave in enumerate(waves):
            wave_raw = zhinst.utils.convert_awg_waveform(wave[0],wave[1])
            self.daq.setVector(f'/{self.device}/awgs/{self.awg_index}/waveform/waves/{i}', wave_raw)
            
    
    def _configure_ct_output(self, ct):
        """configure the entries written to the command table to be valid for the two cases 
            with and without IQ modulation
        
        Parameters
        ----------
        ct: dict
            command table as dictionary
        """
        for entry in ct:
            if 'waveform' in entry.keys():
                if self.iq_modulation:
                    entry['waveform']['awgChannel0'] = ['sigout0','sigout1']
                    entry['waveform']['awgChannel1'] = ['sigout0','sigout1']
                else:
                    entry['waveform']['awgChannel0'] = ['sigout0']
                    entry['waveform']['awgChannel1'] = ['sigout1']
    
    
    def load_ct(self, ct, debug = False):
        """Upload a command table to the AWG
        
        Parameters
        ----------
        ct: dict
            The Command Table, as dictonary
        """
        # put into proper format for IQ output
        self._configure_ct_output(ct)

        # Create Command Table 
        ct_all = {'header': {'version':'0.2'}, 
                  'table': ct}
        node = f"/{self.device:s}/awgs/{self.awg_index}/commandtable/data"
        self.daq.setVector(node, json.dumps(ct_all))
        
        # debug print of ct as written to device
        if debug:
            print(self.awg_index, self.daq.get(node,flat=True)[node][0]['vector'])

        
    def get_ct(self):
        """return the command table which is loaded on the AWG
        
        Returns
        ----------
        ct: dict
            The Command Table, as dictonary
        """
        ct_loaded = self.daq.getInt(f"/{self.device:s}/awgs/{self.awg_index}/commandtable/status")
        if ct_loaded:
            node = f"/{self.device:s}/awgs/{self.awg_index}/commandtable/data"
            ct_raw = self.daq.get(node, flat=True)[node][0]['vector']

            #HACK: wrong format in CT reply from the HDAWG
            ct_raw = ct_raw.replace('"awgChannel0:','"awgChannel0":')
            ct_raw = ct_raw.replace('"awgChannel1:','"awgChannel1":')

            ct = json.loads(ct_raw)
            return ct['table']
        else:
            return None
        
    
    @property
    def iq_modulation(self):
        return self._iq_modulation_enabled

    @iq_modulation.setter 
    def iq_modulation(self, enable):
        """
        enable IQ modulation - proper phase relationship for IQ upconversion
        
        Parameters
        ----------
        enable: int
            enable or disable IQ modulation
        """
        self._iq_modulation_enabled = enable

        # Use the same oscillator for I and Q
        self.daq.setInt(f'/{self.device}/sines/{self.awg_index}/oscselect', self.awg_index)
        self.daq.setInt(f'/{self.device}/sines/{self.awg_index+1}/oscselect', self.awg_index)

        # set correct phase relationship
        self.daq.setDouble(f'/{self.device}/sines/{self.awg_index}/phaseshift', 0.0)
        self.daq.setDouble(f'/{self.device}/sines/{self.awg_index+1}/phaseshift', 90.0)

        # Work with first harmonic
        self.daq.setInt(f'/{self.device}/sines/{self.awg_index}/harmonic', 1)
        self.daq.setInt(f'/{self.device}/sines/{self.awg_index+1}/harmonic', 1)
        
        # set gains
        self.daq.setDouble(f'/{self.device}/awgs/{self.awg_index}/outputs/0/gains/0', 1.0)
        self.daq.setDouble(f'/{self.device}/awgs/{self.awg_index}/outputs/1/gains/1', 1.0)

        if enable:
            # create the correct IQ setting to use with an ideal IQ mixer for SSB modulation
            # the output with these setting will be
            # Out1 = AWG1 * cos(ωt) + AWG2 * sin(ωt)
            # Out2 = -AWG1 * sin(ωt) + AWG2 * cos(ωt)
            self.daq.setInt(f'/{self.device}/awgs/{self.awg_index}/outputs/0/modulation/mode', 3)
            self.daq.setInt(f'/{self.device}/awgs/{self.awg_index}/outputs/1/modulation/mode', 4)

            self.daq.setDouble(f'/{self.device}/awgs/{self.awg_index}/outputs/0/gains/1', -1.0)
            self.daq.setDouble(f'/{self.device}/awgs/{self.awg_index}/outputs/1/gains/0', 1.0)
        else:
            self.daq.setInt(f'/{self.device}/awgs/{self.awg_index}/outputs/0/modulation/mode', 0)
            self.daq.setInt(f'/{self.device}/awgs/{self.awg_index}/outputs/1/modulation/mode', 0)
            
            self.daq.setDouble(f'/{self.device}/awgs/{self.awg_index}/outputs/0/gains/1', 1.0)
            self.daq.setDouble(f'/{self.device}/awgs/{self.awg_index}/outputs/1/gains/0', 1.0)

        # reload the CT if needed - IQ modulated pulses need two entries
        ct = self.get_ct()
        if ct:
            self.load_ct(ct)
            
        
    def set_frequency(self, freq=10000000, osc=0):
        """
        set frequency of internal oscillator
        
        Parameters
        ----------
        freq: double
            Frequency to be set
        osc: int
            Index of oscillator to be changed
        """  
        oscControl = self.daq.getInt(f'/{self.device:s}/system/awg/oscillatorcontrol')
        if oscControl != 1:
            self.oscControl(1)
        
        daq.setDouble(f'/{self.device:s}/oscs/{osc}/freq', freq)
        

# Randomized Benchmarking with HDAWG

## Clifford gates

#### amplitudes (in units of max amplitude) and length (in s) for basic gates 

In [ ]:
pi_amplitude = 1.0
pi_length = 240e-9
pi2_amplitude = 0.5
pi2_length = 240e-9

#### define an envelope function for single qubit gates - here: Gaussian with width = length / 3

In [ ]:
def pulse_envelope(amplitude, length, phase, sigma=1/3, sample_rate=2.4e9, tol=15):
    #sigma = 1/3
    # ensure waveform length is integer multiple of 16
    samples = round(sample_rate * length / 16) * 16
    x = np.linspace(-1, 1, samples)
    # output is complex, where phase later determines the gate rotation axis
    y = amplitude * np.exp(-x**2 / sigma**2 + 1j * np.deg2rad(phase))
    
    return y.round(tol)

#### build the clifford gates out of the elementary single qubit gates

In [ ]:
# all elements of the Clifford group, according to defintion in arXiv:1410.2338
clifford_params = [
    ['I'],
    ['Y/2', 'X/2'],
    ['-X/2', '-Y/2'],
    ['X'],
    ['-Y/2', '-X/2'],
    ['X/2', '-Y/2'],
    ['Y'],
    ['-Y/2', 'X/2'],
    ['X/2', 'Y/2'],
    ['X', 'Y'],
    ['Y/2', '-X/2'],
    ['-X/2', 'Y/2'],
    ['Y/2', 'X'],
    ['-X/2'],
    ['X/2', '-Y/2', '-X/2'],
    ['-Y/2'],
    ['X/2'],
    ['X/2', 'Y/2', 'X/2'],
    ['-Y/2', 'X'],
    ['X/2', 'Y'],
    ['X/2', '-Y/2', 'X/2'],
    ['Y/2'],
    ['-X/2', 'Y'],
    ['X/2', 'Y/2', '-X/2']
]

clifford_len = len(clifford_params)

# parameters of basic single qubit pulses
pulses_params = {
    'I': {'amplitude':0.0, 'length': pi_length, 'phase': 0.0},
    'X': {'amplitude':pi_amplitude, 'length': pi_length, 'phase': 0.0},
    'Y': {'amplitude':pi_amplitude, 'length': pi_length, 'phase': 90.0},
    'X/2': {'amplitude':pi2_amplitude, 'length': pi2_length, 'phase': 0.0},
    'Y/2': {'amplitude':pi2_amplitude, 'length': pi2_length, 'phase': 90.0},
    '-X/2': {'amplitude':pi2_amplitude, 'length': pi2_length, 'phase': 0.0-180.0},
    '-Y/2': {'amplitude':pi2_amplitude, 'length': pi2_length, 'phase': 90.0-180.0},
}

# calculate complex waveforms for single qubit elementary pulses
pulses_waves = {pulse_type: pulse_envelope(**pulse_param) for (pulse_type, pulse_param) in pulses_params.items()}
# calculate complex waveforms for each of the Clifford gates
clifford_waves = [np.concatenate([pulses_waves[i] for i in clifford_gate]) for clifford_gate in clifford_params]

# divide real and complex part of waveforms into I and Q channel outputs
clifford_waves_real = [(np.real(wave), np.imag(wave)) for wave in clifford_waves]

#### basic definitions to manipulate Clifford gates - needed for recovery gate calculation

In [ ]:
def pauli(ind = 'x'):
    """pauli matrices
    """
    if ind =='x':
        res = np.array([[0,1], [1,0]])
    if ind =='y':
        res = np.array([[0,-1j], [1j,0]])
    if ind =='z':
        res = np.array([[1,0], [0,-1]])
        
    return res

def rot_matrix(angle=np.pi, axis='x'):
    """general definition of rotation unitary for a single qubit
    """
    return expm(-1j * angle / 2 * pauli(axis))

def mult_gates(gates, use_linalg=False, tol=20):
    """multiply a variable number of gates / matrices - recursive definition faster for simple 2x2 matrices
    """
    if len(gates) > 1:
        if use_linalg:
            res = np.linalg.multi_dot(gates)
        else:
            res = np.matmul(gates[0], mult_gates(gates[1:], use_linalg=False, tol=tol))
    elif len(gates) == 1:
        res = gates[0]
    
    return res.round(tol)

# generate matrix representation of all Clifford gates from elementary gates
elem_gates = {'I': np.array([[1,0],[0,1]]),
              'X': rot_matrix(np.pi, 'x'),
              'Y': rot_matrix(np.pi, 'y'),
              'X/2': rot_matrix(np.pi / 2, 'x'),
              'Y/2': rot_matrix(np.pi / 2, 'y'),
              '-X/2': rot_matrix(-np.pi / 2, 'x'),
              '-Y/2': rot_matrix(-np.pi / 2, 'y')}

clifford_matrices = [[elem_gates[gate] for gate in gates] for gates in clifford_params]
clifford_gates = [mult_gates(matrices) for matrices in clifford_matrices]

def glob_phase(phase, dim=2):
    """global phase operator for dimensionality dim
    """
    return np.exp(1j * phase) * np.identity(dim)

def match_up_to_phase(target, gates, dim=2, verbose=False):
    """finds the element of the list gates that best matches the target gate up to a global phase of integer multiples of pi
    """
    # set of global phase operators for integer multiples of pi
    glob_phases = [glob_phase(0, dim), glob_phase(np.pi, dim)]
    # gates up to global phases
    gates_2 = [[mult_gates([gate1, gate2]) for gate2 in glob_phases] for gate1 in gates]
    # index of gate that is closest to target up to global phase (using frobenius norm)
    match_index = np.argmin([np.amin([np.linalg.norm(target - gate) for gate in gates]) for gates in gates_2])
    
    return match_index

#### function to calculate the last gate in the sequence - recovery gate which leads back to initial state (up to global phase)

In [ ]:
def calculate_inverse_clifford(clifford_list, gates=clifford_gates):
    """Calculates the final recovery clifford gate

    Parameters:
    clifford_list: list
        a list containing the indices of the clifford sequence to be inverted
    gate: list
        a list containing the gates to compare the recovery gate to
    """
    # matrix representation of Clifford sequence
    seq_gate = mult_gates([gates[gate] for gate in clifford_list])
    # recovery gate - inverse of full sequence
    rec_gate = np.linalg.inv(seq_gate)
    # index of recovery gate (up to global phase)
    recovery = int(match_up_to_phase(rec_gate, clifford_gates))
    
    return recovery

## connect to the AWG - here: signal played on first two channels, connected to dataserver via ethernet / usb

In [ ]:
awg = HDAWG_Core(daq, dev_hd, awg_index=0, connect_type='1gbe')
# awg = HDAWG_Core(daq, dev_hd, awg_index=0, connect_type='usb')

## Random command table approach

This method determines each random RB sequence as a list of elements of the Clifford group each sequence contains, (not the full waveform) and uploads this sequence to the command table. seqC is uploaded and compiled only once, as is the waveform data.
Limited to sequence length of 1024 elements, due to the size of the command table on the HD

#### function to generate a command table for each random sequence

In [ ]:
def generate_ct_entry(i, gate):
    return {
              "index": i,
              "waveform": {
                 "index": gate,
              }
    }

#### define the seqC program as string

In [ ]:
# how many averages / repetitions of each sequence
num_Averages = 2**0

# Waveform definition - allocating the waveform memory for the seqC program
waveforms_def = ""
for i,wave in enumerate(clifford_waves):
    wave_len = len(wave)
    waveforms_def += f"assignWaveIndex(placeholder({wave_len:d}), placeholder({wave_len:d}), {i:d});\n"
    
# define register index for input of sequence length
AWG_REGISTER_M = 0

# define the seqC program as string
hd_rb_program_0 = f"""
//Waveforms definition
{waveforms_def:s}

//sequence length from user register
var m = getUserReg({AWG_REGISTER_M:d});

// send a trigger at start of sequence
setTrigger(3);
wait(5);
setTrigger(0);

repeat ({num_Averages}) {{
    //Reset the oscillator phase
    resetOscPhase();
    wait(5);

    //execute random sequence by stepping through the command table
    var i;
    for (i = 0; i < m; i++) {{
      executeTableEntry(i);
    }}
}}
"""

#### configure the AWG, upload the seqC program and the waveform data

In [ ]:
awg.config(hd_rb_program_0, waves=clifford_waves_real)

# enable iq modulation or generate simple quadrature pulses for illustration
awg.iq_modulation = True
# set digital oscillator frequency to 15 MHz (IF)
awg.set_frequency(freq=15000000, osc=0)

#### run RB with random sequences generated locally, and uploaded to device as command table

In [ ]:
# number of different sequence lengths
n = 10
# number of different random sequences per length
k = 20

# set the AWG to a known state
awg.stop()

start = time.perf_counter_ns()
upload_time = []
exec_time = []

# iterate over sequence lengths
for len_exp in range(1,n+1):
    # define sequence length = 2^n
    M = 2**len_exp
    # HACK: command table length limited to 1024 entries
    if M > 1023:
        M = 1023

    # iterate over different random sequences
    for rand_i in range(k):
        
        exec_start = time.perf_counter_ns()
        
        # Generate a RB sequence as a sequence of random Clifford indices
        gates_M1 = [random.randrange(0,24) for i in range(M)]
        # find recovery gate
        gate_M = calculate_inverse_clifford(gates_M1)
        # full sequence
        gates_M = gates_M1 + [gate_M]
    
        # generate command table correspoding to the reandom sequence
        ct = [generate_ct_entry(i, gate) for i, gate in enumerate(gates_M)]
        
        up_start = time.perf_counter_ns()
        
        # upload command table to device
        awg.load_ct(ct)
        # tell AWG how many entries are in command table via user register
        awg.setUserReg(AWG_REGISTER_M, M+1)
        
        upload_time.append(time.perf_counter_ns() - up_start)
        
        # Start the HDAWG AWG sequencer, wait for execution to end
        awg.run(wait=True)
        
        exec_time.append(time.perf_counter_ns() - exec_start)

tot_time = time.perf_counter_ns() - start

upload_time = np.array(upload_time) * 1e-9
exec_time = np.array(exec_time) * 1e-9

print(f'Total time: {tot_time*1e-9:.3f}')
print(f'Per iteration time: {tot_time/(k*n)*1e-9:.3f}s')

In [ ]:
plt.plot(upload_time, 'r', label='upload time')
plt.plot(exec_time, 'b', label='execution time')
plt.legend()
plt.show()

## PRNG approach

For each random sequency, we transfer only a sequence length, a random seed and the index of the final recovery Clifford gate. seqC is uploaded and compiled only once. The command table contains all clifford gates and is set once at the start, together with waveform data.
This program precalculates the RB sequence using a model of the HDAWGs PRNG and determines the correct recovery gate, but the HDAWG generates the random sequence locally. Sequence length can be up to 2**16 + 1 (limited by the possible internal states of the PRNG)

#### A computational representation of the PRNG on the HDAWG

In [ ]:
class HDAWG_PRNG:
    def __init__(self, seed=0xcafe, lower=0, upper=2**16-1):
        self.lsfr = seed
        self.lower = lower
        self.upper = upper

    def next(self):
        lsb = self.lsfr & 1
        self.lsfr = self.lsfr >> 1
        if (lsb):
            self.lsfr = 0xb400 ^ self.lsfr
        rand = ((self.lsfr * (self.upper-self.lower+1) >> 16) + self.lower) & 0xffff
        return rand

#### define the seqC program and the command table

In [ ]:
# how many averages / repetitions of each sequence
num_Averages = 2**0

# Waveform definition - allocating the waveofmr memory for the seqC program
waveforms_def = ""
for i,wave in enumerate(clifford_waves):
    wave_len = len(wave)
    waveforms_def += f"assignWaveIndex(placeholder({wave_len:d}), placeholder({wave_len:d}), {i:d});\n"
    
# define user registers that will be used in the seqC
AWG_REGISTER_M1 = 0
AWG_REGISTER_SEED = 1
AWG_REGISTER_RECOVERY = 2
AWG_REGISTER_START = 3

# define the seqC program to be uploaded 
hd_rb_program_1 = f"""
const CLIFFORD_LEN = 24;

//Waveform definition
{waveforms_def:s}

// Runtime parameters, set by user registers:
// sequence length
var m1 = getUserReg({AWG_REGISTER_M1:d});
// PRNG seed
var seed = getUserReg({AWG_REGISTER_SEED:d});
// recovery gate index
var recovery = getUserReg({AWG_REGISTER_RECOVERY:d});

//Configure the PRNG
setPRNGSeed(seed);
setPRNGRange(0, CLIFFORD_LEN);

// trigger at start of each sequence
setTrigger(1);
wait(5);
setTrigger(0);

repeat ({num_Averages}) {{
    //Reset the oscillator phase
    resetOscPhase();
    wait(5);

    // (Pseudo) Random sequence of command table entries
    repeat (m1) {{
        var gate = getPRNGValue();
        executeTableEntry(gate);
    }}
    // Final recovery gate
    executeTableEntry(recovery);
}}
"""

# define the ct table, where each element is just one of the Clifford gates
ct = []
for i in range(clifford_len):
    entry = {
              "index": i,
              "waveform": {
                 "index": i
              }
    }        
    ct.append(entry)

#### configure the AWG, upload and compile the seqC program and upload the command table and waveform data

In [ ]:
# awg = HDAWG_Core(daq, dev_hd, 0, connect_type = '1gbe')
awg.config(hd_rb_program_1, ct=ct, waves=clifford_waves_real)

# for demonstration purposes set IQ modulation to False, for real IQ pulses set to True
awg.iq_modulation = True
# set digital oscillator frequency to 15 MHz (IF)
awg.set_frequency(freq=15000000, osc=0)

#### run Randomized Benchmarking with each random sequence generated on the HD itself

In [ ]:
# n different sequence lengths
n = 10
# k different random sequences per length
k = 20

# set the AWG to a known state
awg.stop()

start = time.perf_counter_ns()
upload_time = []
exec_time = []

# iterate over sequence lengths
for len_exp in range(1, n+1):
    # length of sequence is 2**n
    M = 2**len_exp
    
    # iterate over different random sequences
    for rand_i in range(k):
        
        exec_start = time.perf_counter_ns()
        
        # generate random seed for the PRNG
        seed = random.randrange(1, 2**16-1)
        # use seed and PRNG model to determine gate sequence that will be played on AWG
        prng = HDAWG_PRNG(seed=seed,lower=0,upper=clifford_len-1)
        gates_M1 = [prng.next() for i in range(M)]
        # calculate the recovery clifford gate
        gate_M = calculate_inverse_clifford(gates_M1)
        
        up_start = time.perf_counter_ns()
        
        # set AWG registers to transmit information on 
        # - sequence length
        # - PRNG seed
        # - last clifford gate in sequence
        registers = [
            (AWG_REGISTER_M1, M),
            (AWG_REGISTER_SEED, seed),
            (AWG_REGISTER_RECOVERY, gate_M)
        ]
        awg.setUserRegisters(registers)
        
        upload_time.append(time.perf_counter_ns() - up_start)

        # Start the HDAWG AWG sequencer, wait for execution to end
        awg.run(wait=True)
        
        exec_time.append(time.perf_counter_ns() - exec_start)

upload_time = np.array(upload_time) * 1e-9
exec_time = np.array(exec_time) * 1e-9
        
tot_time = time.perf_counter_ns() - start
print(f'Total time: {tot_time*1e-9:.3f}s')
print(f'Per iteration time: {tot_time/(k*n)*1e-9:.3f}s')

In [ ]:
plt.plot(upload_time, 'r', label='upload time')
plt.plot(exec_time, 'b', label='execution time')
plt.legend()
plt.show()

## Traditional methods - upload the full sequence

#### generate full two-channel waveform from Clifford definition - including digital modulation, if required

In [ ]:
def generate_wave(clifford_sequence, freq=10e6, iq_modulation=True, pad_zero=None, sample_rate=2.4e9):
    # wave_real - zero for Y-pulse, non-zero for X-pulse - negative amplitude flips by pi
    wave_real = np.concatenate([clifford_waves_real[i][0] for i in clifford_sequence])
    # wave_imag - zero for X-pulse, non-zero for Y-pulse - negative amplitude flips by pi
    wave_imag = np.concatenate([clifford_waves_real[i][1] for i in clifford_sequence])
    
    if pad_zero is not None:
        pad_len = pad_zero - wave_real.size
        if pad_len > 0:
            wave_real = np.pad(wave_real, (0, pad_len), 'constant', constant_values=0)
            wave_imag = np.pad(wave_imag, (0, pad_len), 'constant', constant_values=0)
    
    wave_len = wave_real.size
        
    if iq_modulation:
        wave_time = wave_len / sample_rate
        wave_sin = np.sin(2 * np.pi * freq * np.linspace(0, wave_time, wave_len))
        wave_cos = np.cos(2 * np.pi * freq * np.linspace(0, wave_time, wave_len))
        # X-pulse: sin on I, cos on Q / Y-pulse: -cos on I, sin on Q
        wave_I = wave_sin * wave_real + wave_cos * wave_imag
        wave_Q = -wave_cos * wave_real + wave_sin * wave_imag
    else:
        wave_I = wave_real
        wave_Q = wave_imag
        
    return [(wave_I, wave_Q)]    

In [ ]:
## test of wave generating code

m=4
# Generate a RB sequence as a sequence of random Clifford indices
gates_m1 = [random.randrange(0,24) for i in range(m)]
# find recovery gate
gate_m = calculate_inverse_clifford(gates_m1)
# full sequence
gates_m = gates_m1 + [gate_m]

test_wave0 = generate_wave(gates_m, iq_modulation=False, pad_zero=8000)
plt.plot(test_wave0[0][0])
plt.plot(test_wave0[0][1])
plt.show()

test_wave1 = generate_wave(gates_m, iq_modulation=True, pad_zero=8000)
plt.plot(test_wave1[0][0])
plt.plot(test_wave1[0][1])
plt.show()

#### single entry command table - only employed for simplicity of the seqC program

In [ ]:
ct_simple = [{
              "index": 0,
              "waveform": {
                 "index": 0,
              }
            }]

#### define the seqC program as string, with length of waveform as parameter

In [ ]:
# number of averages / repetitions
num_Averages = 2**0

# define the seqC program as string
def hd_rb_program_2(wave_len, num_Averages=num_Averages): 
    res = f"""
    //Waveform definition - single, two-channel waveform
    assignWaveIndex(placeholder({wave_len:d}), placeholder({wave_len:d}), 0);

    // send a trigger at start of sequence
    setTrigger(3);
    wait(5);
    setTrigger(0);
    
    repeat ({num_Averages}) {{
        //Reset the oscillator phase
        resetOscPhase();
        wait(5);

        //execute random sequence from single command table entry
        executeTableEntry(0);
    }}
    """

    return res

### pre-generate and upload full waveform including modulation - variable waveform length, requiring recompilation at every step

In [ ]:
# number of different sequence lengths
n = 10
# number of different random sequences per length
k = 20
# include IQ_modulation?
iq_modulation = True
freq = 10e6

# set the AWG to a known state
awg.stop()

# start = time.time()
start = time.perf_counter_ns()
upload_time = []
exec_time = []

# iterate over sequence lengths
for len_exp in range(1,n+1):
    # define sequence length = 2^n
    M = 2**len_exp

    # iterate over different random sequences
    for rand_i in range(k):
        
        exec_start = time.perf_counter_ns()
        
        # Generate a RB sequence as a sequence of random Clifford indices
        gates_M1 = [random.randrange(0,24) for i in range(M)]
        # find recovery gate
        gate_M = calculate_inverse_clifford(gates_M1)
        # full sequence
        gates_M = gates_M1 + [gate_M]
        
        wave_all = generate_wave(gates_M, freq=freq, iq_modulation=iq_modulation)
        wave_len = wave_all[0][0].size
        
        up_start = time.perf_counter_ns()
        
        # upload and compile seqC, waveform and command table
        awg.config(hd_rb_program_2(wave_len), waves=wave_all, ct=ct_simple)
                
        upload_time.append(time.perf_counter_ns() - up_start)
        
        # Start the HDAWG AWG sequencer, wait for execution to end
        awg.run(wait=True)
        
        exec_time.append(time.perf_counter_ns() - exec_start)

tot_time = time.perf_counter_ns() - start

upload_time = np.array(upload_time) * 1e-9
exec_time = np.array(exec_time) * 1e-9

print(f'Total time: {tot_time*1e-9:.3f}s')
print(f'Per iteration time: {tot_time/(k*n)*1e-9:.3f}s')

In [ ]:
plt.plot(upload_time, 'r', label='upload time')
plt.plot(exec_time, 'b', label='execution time')
plt.legend()
plt.show()

### pre-generate and upload full waveform including modulation - preallocate max waveform length, recompilation required for every sequence length

In [ ]:
# number of different sequence lengths
n = 10
# number of different random sequences per length
k = 20
# include IQ_modulation?
iq_modulation = True
freq = 15e6

# length of single gate in samples
pulse_len = round(2.4e9 * pi_length / 16) * 16

# set the AWG to a known state
awg.stop()

# start = time.time()
start = time.perf_counter_ns()
upload_time = []
exec_time = []

# iterate over sequence lengths
for len_exp in range(1,n+1):
    # define sequence length = 2^n
    M = 2**len_exp

    # waveform is preallocated with maximum possible length for given sequence length 
    wave_len = (M + 1) * 3 * pulse_len
    
    # upload and compile seqC and command table
    awg.config(hd_rb_program_2(wave_len))
    awg.load_ct(ct=ct_simple)
    
    # iterate over different random sequences
    for rand_i in range(k):
        
        exec_start = time.perf_counter_ns()
        
        # Generate a RB sequence as a sequence of random Clifford indices
        gates_M1 = [random.randrange(0,24) for i in range(M)]
        # find recovery gate
        gate_M = calculate_inverse_clifford(gates_M1)
        # full sequence
        gates_M = gates_M1 + [gate_M]
        
        # generate waveform, pad with zeros to max length
        wave_all = generate_wave(gates_M, freq=freq, iq_modulation=iq_modulation, pad_zero=wave_len)
        
        up_start = time.perf_counter_ns()
        
        # upload waveform
        awg.load_waves(waves=wave_all)
                        
        upload_time.append(time.perf_counter_ns() - up_start)
        
        # Start the HDAWG AWG sequencer, wait for execution to end
        awg.run(wait=True)
        
        exec_time.append(time.perf_counter_ns() - exec_start)

tot_time = time.perf_counter_ns() - start

upload_time = np.array(upload_time) * 1e-9
exec_time = np.array(exec_time) * 1e-9

print(f'Total time: {tot_time*1e-9:.3f}')
print(f'Per iteration time: {tot_time/(k*n)*1e-9:.3f}s')

In [ ]:
plt.plot(upload_time, 'r', label='upload time')
plt.plot(exec_time, 'b', label='execution time')
plt.legend()
plt.show()

### pre-generate and upload full waveform including modulation - preallocate max waveform length for all sequences, no recompilation required

In [ ]:
# number of different sequence lengths
n = 10
# number of different random sequences per length
k = 20
# include IQ_modulation?
iq_modulation = True
freq = 10e6

# length of single gate in samples
pulse_len = round(2.4e9 * pi_length / 16) * 16

# set the AWG to a known state
awg.stop()

# start = time.time()
start = time.perf_counter_ns()
upload_time = []
exec_time = []

# waveform is preallocated with maximum possible length for all sequence lengths 
wave_len = (2**n + 1) * 3 * pulse_len

# upload and compile seqC and command table
awg.config(hd_rb_program_2(wave_len))
awg.load_ct(ct=ct_simple)

# iterate over sequence lengths
for len_exp in range(1, n+1):
    # define sequence length = 2^n
    M = 2**len_exp

    # iterate over different random sequences
    for rand_i in range(k):
        
        exec_start = time.perf_counter_ns()
        
        # Generate a RB sequence as a sequence of random Clifford indices
        gates_M1 = [random.randrange(0,24) for i in range(M)]
        # find recovery gate
        gate_M = calculate_inverse_clifford(gates_M1)
        # full sequence
        gates_M = gates_M1 + [gate_M]
        
        # generate waveform, pad with zeros to max length
        wave_all = generate_wave(gates_M, freq=freq, iq_modulation=iq_modulation, pad_zero=wave_len)
        
        up_start = time.perf_counter_ns()
        
        # upload waveform
        awg.load_waves(waves=wave_all)
                        
        upload_time.append(time.perf_counter_ns() - up_start)
        
        # Start the HDAWG AWG sequencer, wait for execution to end
        awg.run(wait=True)
        
        exec_time.append(time.perf_counter_ns() - exec_start)

tot_time = time.perf_counter_ns() - start

upload_time = np.array(upload_time) * 1e-9
exec_time = np.array(exec_time) * 1e-9

print(f'Total time: {tot_time*1e-9:.3f}')
print(f'Per iteration time: {tot_time/(k*n)*1e-9:.3f}s')

In [ ]:
plt.plot(upload_time, 'r', label='upload time')
plt.plot(exec_time, 'b', label='execution time')
plt.legend()
plt.show()